In [6]:
import torchgan

In [9]:
import os
import random
import matplotlib.pyplot as plt
import numpy as np
# Pytorch and Torchvision Imports
import torch
import torchvision
from torch.optim import Adam
import torch.nn as nn
import torch.utils.data as data
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torchvision.utils as vutils
# Torchgan Imports
import torchgan
from torchgan.layers import SpectralNorm2d, SelfAttention2d
from torchgan.models import Generator, Discriminator
from torchgan.losses import WassersteinGeneratorLoss, WassersteinDiscriminatorLoss, WassersteinGradientPenalty
from torchgan.trainer import Trainer


In [ ]:
def ResUp(nn.Module):
    def __init__(self, step_channels=64):
        d = step_channels
        self.skip = nn.Sequential(
                      nn.Upsample(),
                      nn.Conv2d(d * 2, d * 4, 1))
        self.model = nn.Sequential(
                    nn.BatchNorm2d(d), nn.LeakyReLU(0.2),
                    nn.Upsample(),
                    nn.Conv2d(d * 2, d * 4, 3)
                    nn.BatchNorm2d(d * 2), nn.LeakyReLU(0.2),
                    nn.Conv2d(d * 2, d * 4, 3))
    def forward(self, x):
        main = self.model(x)
        skip = self.skip(x)
        out = torch.add(main, skip)
        return out


In [ ]:
def ResDown(nn.Module):
    def __init__(self, step_channels=64):
        d = step_channels
        self.skip = nn.Sequential(
                      nn.Conv2d(d * 2, d * 4, 1),
                      nn.AvgPool2d())
        self.model = nn.Sequential(
                    nn.LeakyReLU(0.2),
                    nn.Conv2d(d * 2, d * 4, 3),
                    nn.LeakyReLU(0.2),
                    nn.Conv2d(d * 2, d * 4, 3),
                    nn.AvgPool2d())
    def forward(self, x):
        main = self.model(x)
        skip = self.skip(x)
        out = torch.add(skip, main)
        return out

In [ ]:
class BigGanGenerator(Generator):
    def __init__(self, step_channels=64):
        self.linear = SpectralNorm2d(nn.Linear(4 * 4 * 16 * chn))
        self.conv = nn.ModuleList([ResUp(16*chn, 16*chn),
                                ResUp(16*chn, 8*chn),
                                ResUp(8*chn, 4*chn),
                                ResUp(4*chn, 2*chn),
                                SelfAttention2d(2*chn),
                                ResUp(2*chn, 1*chn)])
        self.last = nn.Sequential(nn.BatchNorm2d(d), nn.LeakyReLU(0.2),
                nn.Conv2d(d * 2, 3, 3), nn.Tanh())
    def forward(self, x):
        out = self.linear(x)
        out = self.conv(out)
        out = self.last(out)
        return out